In [4]:
from clickhouse_driver import Client
import pandas as pd

ModuleNotFoundError: No module named 'clickhouse_driver'

In [21]:
import os
from datetime import datetime, timedelta
import pandas as pd
df = pd.read_csv('./processed_log.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['timediff'] = df['timediff'].fillna(0)
# сгруппируем данные по пользователю
max_counter = 1

# Сортируем данные по пользователям и времени
df = df.sort_values(['userid', 'timestamp'])

# Создаем группы для каждого пользователя
groups = df.groupby('userid')

# Инициализируем новый столбец counter_of_sessions с нулями
df['counter_of_sessions'] = 1

# Проходимся по каждой группе и создаем новую сессию,
# когда между действиями прошло более 30 секунд
for user, group in groups:
    session_counter = max_counter
    prev_time = None  # Объявление prev_time здесь

    for index, row in group.iterrows():
        if prev_time is None:
            prev_time = row['timestamp']
            continue

        timediff = (row['timestamp'] - prev_time).total_seconds()
        if timediff > 30:
            session_counter += 1

        prev_time = row['timestamp']
        df.loc[index, 'counter_of_sessions'] = session_counter

    max_counter = session_counter + 1


df.head(10)
# last_action_time = df['timestamp'].max()

# # Разделяем сессии на две группы: активные и неактивные
# active_sessions = []
# inactive_sessions = []
# for session_id, group in df.groupby('counter_of_sessions'):
#     last_session_action_time = group['timestamp'].max()
#     time_diff = last_action_time - last_session_action_time
#     if time_diff < timedelta(minutes=30):
#         active_sessions.append(group)
#     else:
#         inactive_sessions.append(group)

# # Сгруппируем данные по counter_of_sessions
# active_df = pd.concat(active_sessions).loc[:,
#             ['userid', 'sessionid', 'from', 'timestamp', 'action', 'value', 'category', 'age', 'gender', 'city']]
# if os.path.exists('/home/chessnok/airflow/log.csv'):
#     os.remove('/home/chessnok/airflow/log.csv')
# active_df.to_csv('/home/chessnok/airflow/log.csv', index=False)
# save_path = '/home/chessnok/airflow/input.csv'
# client = Client(host="ll59i07mv8.eu-central-1.aws.clickhouse.cloud", user="default", password="cqVcCWZ~xl98V", port=9440, secure=True)
# client.execute(
#     'CREATE TABLE IF NOT EXISTS tmp_input (userid String, sessionid Int64, from Nullable(String), timestamp DateTime, '
#     'action String, value Float64, category String, age Int64, gender String, city String, time_of_action '
#     'DateTime, time_of_next_action DateTime, timediff Int64, counter_of_sessions Int64) ENGINE = MergeTree() '
#     'ORDER BY timestamp;')
# if len(inactive_sessions) != 0:
#     print(len(inactive_sessions))
#     inactive_df = pd.concat(inactive_sessions)
#     inactive_df.drop("time_diff", axis=1, inplace=True)
#     inactive_df.to_csv(save_path, index=False)
#     client.insert_dataframe(inactive_df, 'tmp_input')
    # inactive_df['timestamp'] = inactive_df['timestamp'].apply(
    #     lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    # client.command('INSERT INTO tmp_input'
    #                ' '
    #                'VALUES', inactive_df.to_dict('records'))


,userid,sessionid,from,timestamp,action,value,category,age,gender,city,time_of_next_action,timediff,counter_of_sessions
0,user_10000,593,NaN,2023-04-17 20:40:25,mainpage,0,NaN,18,М,Москва,2023-04-17 20:40:58,0.55,1
1,user_10000,593,NaN,2023-04-17 20:40:58,mainpage,0,NaN,18,М,Москва,2023-04-17 20:41:00,0.03,2
2,user_10000,593,NaN,2023-04-17 20:41:00,mainpage,0,NaN,18,М,Москва,2023-04-17 20:41:41,0.68,2
3,user_10000,593,NaN,2023-04-17 20:41:41,mainpage,0,NaN,18,М,Москва,2023-04-17 20:41:55,0.23,3
4,user_10000,593,NaN,2023-04-17 20:41:55,mainpage,0,NaN,19,М,Москва,2023-04-17 20:43:23,1.47,3
5,user_10000,593,NaN,2023-04-17 20:43:23,mainpage,0,test,19,М,Москва,2023-04-17 20:43:57,0.57,4
6,user_10000,593,NaN,2023-04-17 20:43:57,mainpage,0,test,19,М,Москва,2023-05-17 09:00:00,42496.05,5
7,user_10000,593,NaN,2023-05-17 09:00:00,mainpage,0,NaN,18,М,Москва,NaN,0.00,6
8,user_10010,593,NaN,2023-12-16 21:00:00,mainpage,0,NaN,18,М,Москва,NaN,0.00,1


In [13]:
active_df.head(5)

,userid,sessionid,from,timestamp,action,value,category,age,gender,city,time_of_next_action,timediff,time_diff,counter_of_sessions
8,user_10010,593,NaN,2023-12-16 21:00:00,mainpage,0,NaN,18,М,Москва,NaN,NaN,213 days 12:00:00,3


In [10]:
inactive_df.head()

,userid,sessionid,from,timestamp,action,value,category,age,gender,city,time_of_next_action,timediff,time_diff,counter_of_sessions
0,user_10000,593,NaN,2023-04-17 20:40:25,mainpage,0,NaN,18,М,Москва,2023-04-17 20:40:58,0 days 00:00:33,NaN,1
1,user_10000,593,NaN,2023-04-17 20:40:58,mainpage,0,NaN,18,М,Москва,2023-04-17 20:41:00,0 days 00:00:01.800000,0 days 00:00:33,1
2,user_10000,593,NaN,2023-04-17 20:41:00,mainpage,0,NaN,18,М,Москва,2023-04-17 20:41:41,0 days 00:00:40.800000,0 days 00:00:02,1
3,user_10000,593,NaN,2023-04-17 20:41:41,mainpage,0,NaN,18,М,Москва,2023-04-17 20:41:55,0 days 00:00:13.800000,0 days 00:00:41,1
4,user_10000,593,NaN,2023-04-17 20:41:55,mainpage,0,NaN,19,М,Москва,2023-04-17 20:43:23,0 days 00:01:28.200000,0 days 00:00:14,1
